## Embeddings

Cada modelo tiene todos los vídeos y la etiqueta en la columna **shoot_zone**,  donde lanzamiento a la derecha es 0, al centro es 1 y a la izquierda  es 2. 

| Fichero                      | Modelo          | Comentarios sobre “\_SUFIJO”                              |
| ---------------------------- | --------------- | --------------------------------------------------------- |
| **baseline\_CASIAB.csv**     | Baseline        | Versión “base” (p.ej. GEINet simple) entrenada en CASIA-B |
| **baseline\_OUMVLP.csv**     | Baseline        | Igual que el anterior, pero pre-entrenado en OU-MVLP      |
| **gaitgl.csv**               | GaitGL          | GaitGL estándar (dataset por defecto, p.ej. CASIA-B)      |
| **gaitgl\_OUMVLP.csv**       | GaitGL          | Pre-entrenado en OU-MVLP                                  |
| **gaitgl\_GREW\.csv**        | GaitGL          | Pre-entrenado en GREW                                     |
| **gaitgl\_GREW\_BNNeck.csv** | GaitGL + BNNeck | Mismo que el anterior, con cuello de batch-norm extra     |
| **gaitpart.csv**             | GaitPart        | GaitPart estándar                                         |
| **gaitpart\_OUMVLP.csv**     | GaitPart        | Pre-entrenado en OU-MVLP                                  |
| **gaitpart\_GREW\.csv**      | GaitPart        | Pre-entrenado en GREW                                     |
| **gaitset.csv**              | GaitSet         | GaitSet estándar                                          |
| **gaitset\_OUMVLP.csv**      | GaitSet         | Pre-entrenado en OU-MVLP                                  |
| **gaitset\_GREW\.csv**       | GaitSet         | Pre-entrenado en GREW                                     |
| **gln\_phase1.csv**          | GLN (fase 1)    | Primer bloque/fase de extracción del modelo “GLN”         |
| **gln\_phase2.csv**          | GLN (fase 2)    | Fase de refinamiento o bloque final del mismo “GLN”       |


In [1]:
import os
import pandas as pd

# 1. Directorio con los CSV
data_dir = "Gait_Embeddings_good/"

# 2. Listar sólo los archivos .csv
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
print("Archivos encontrados:", csv_files)

# 3. Leer cada CSV en un DataFrame de pandas
dfs = {}
for fname in csv_files:
    path = os.path.join(data_dir, fname)
    dfs[fname] = pd.read_csv(path)

# 4. Explorar cada DataFrame
for name, df in dfs.items():
    print(f"\n=== {name} ===")
    print("Shape:", df.shape)                  # filas × columnas
    print("Columnas:", df.columns.tolist())    # lista de nombres
    print("Primeras 5 filas:")
    print(df.head().to_string(index=False))    # muestra las primeras filas

    # Opcional: ver tipo de datos y memoria
    print("\nInfo:")
    print(df.info())
    print("\nDescripción estadística de columnas numéricas:")
    print(df.describe().T)  # transpuesta para leer mejor


Archivos encontrados: ['baseline_CASIAB.csv', 'baseline_OUMVLP.csv', 'gaitgl.csv', 'gaitgl_GREW.csv', 'gaitgl_GREW_BNNeck.csv', 'gaitgl_OUMVLP.csv', 'gaitpart.csv', 'gaitpart_GREW.csv', 'gaitpart_OUMVLP.csv', 'gaitset.csv', 'gaitset_GREW.csv', 'gaitset_OUMVLP.csv', 'gln_phase1.csv', 'gln_phase2.csv']


C:\Users\grial_h2phao1\AppData\Local\Temp\ipykernel_18012\3135675460.py:15: DtypeWarning: Columns (129) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[fname] = pd.read_csv(path)



=== baseline_CASIAB.csv ===
Shape: (13392, 261)
Columnas: ['step', 'feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48', 'feat_49', 'feat_50', 'feat_51', 'feat_52', 'feat_53', 'feat_54', 'feat_55', 'feat_56', 'feat_57', 'feat_58', 'feat_59', 'feat_60', 'feat_61', 'feat_62', 'feat_63', 'feat_64', 'feat_65', 'feat_66', 'feat_67', 'feat_68', 'feat_69', 'feat_70', 'feat_71', 'feat_72', 'feat_73', 'feat_74', 'feat_75', 'feat_76', 'feat_77', 'feat_78', 'feat_79', 'feat_80', 'feat_81', 'feat_82', 'feat_83', 'feat_84', 'feat_8

## Modelo Perceptrón


In [9]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

# ————————————————
#  CONFIGURACIÓN
# ————————————————
CSV_PATH  = 'Gait_Embeddings_good/baseline_CASIAB.csv'  # Ruta al CSV que quieras probar
POOL_TYPE = 'max'                    # 'mean' o 'max'
TEST_SIZE  = 0.2
BATCH_SIZE = 32
EPOCHS     = 30
LR         = 1e-3
DEVICE     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ————————————————
#  1) Función para cargar y hacer pooling
# ————————————————
def load_and_pool(csv_path, pool='mean'):
    df = pd.read_csv(csv_path)
    feat_cols = [c for c in df.columns if c.startswith('feat_')]  # feat_0 … feat_255
    grouped  = df.groupby('video_ID')

    X_list, y_list = [], []
    for vid, g in grouped:
        arr = g[feat_cols].values  # (T, D)
        if pool == 'mean':
            seq_emb = arr.mean(axis=0)
        elif pool == 'max':
            seq_emb = arr.max(axis=0)
        else:
            raise ValueError("pool debe ser 'mean' o 'max'")
        X_list.append(seq_emb)
        # Asumimos que shoot_zone es constante dentro de cada video_ID
        y_list.append(g['shoot_zone'].iloc[0])

    X = np.vstack(X_list).astype(np.float32)   # (N_sequences, D)
    y = np.array(y_list).astype(np.int64)      # (N_sequences,)
    return X, y

# ————————————————
#  2) Carga + pooling
# ————————————————
X, y = load_and_pool(CSV_PATH, pool=POOL_TYPE)
print(f"Cargadas {X.shape[0]} secuencias, dimensión de embedding = {X.shape[1]}")

# ————————————————
#  3) Train/Test split + escalado
# ————————————————
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, 
    stratify=y, random_state=42
)
scaler    = StandardScaler()
X_train   = scaler.fit_transform(X_train)
X_test    = scaler.transform(X_test)

# TensorDatasets y DataLoaders
train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_ds  = TensorDataset(torch.from_numpy(X_test),  torch.from_numpy(y_test))
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE)

# ————————————————
#  4) Definición del MLP
# ————————————————
D = X.shape[1]
mlp = nn.Sequential(
    nn.Linear(D, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 64),
    nn.ReLU(inplace=True),
    nn.Linear(64, 3)         # 3 clases: 0=derecha,1=centro,2=izquierda
).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp.parameters(), lr=LR)

# ————————————————
#  5) Bucle de entrenamiento
# ————————————————
for epoch in range(1, EPOCHS+1):
    mlp.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        optimizer.zero_grad()
        out = mlp(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)

    # Pérdida media
    train_loss = running_loss / len(train_ds)

    # Evaluación en test
    mlp.eval()
    correct = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            preds = mlp(xb).argmax(dim=1)
            correct += (preds == yb).sum().item()
    test_acc = correct / len(test_ds)

    print(f"Epoch {epoch:02d} | Train loss: {train_loss:.4f} | Test acc: {test_acc:.4f}")

# ————————————————
#  6) Resultado final
# ————————————————
print("\n>>> Entrenamiento completo:")
mlp.eval()
correct = 0
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        correct += (mlp(xb).argmax(dim=1) == yb).sum().item()
print(f"Precisión final en test: {correct/len(test_ds):.4f}")

Cargadas 432 secuencias, dimensión de embedding = 256
Epoch 01 | Train loss: 1.0637 | Test acc: 0.5287
Epoch 02 | Train loss: 0.9552 | Test acc: 0.5172
Epoch 03 | Train loss: 0.8789 | Test acc: 0.5402
Epoch 04 | Train loss: 0.7811 | Test acc: 0.5287
Epoch 05 | Train loss: 0.6828 | Test acc: 0.5517
Epoch 06 | Train loss: 0.5638 | Test acc: 0.5057
Epoch 07 | Train loss: 0.4480 | Test acc: 0.5057
Epoch 08 | Train loss: 0.3371 | Test acc: 0.5057
Epoch 09 | Train loss: 0.2538 | Test acc: 0.5287
Epoch 10 | Train loss: 0.1564 | Test acc: 0.5172
Epoch 11 | Train loss: 0.0997 | Test acc: 0.5172
Epoch 12 | Train loss: 0.0604 | Test acc: 0.5057
Epoch 13 | Train loss: 0.0352 | Test acc: 0.5287
Epoch 14 | Train loss: 0.0226 | Test acc: 0.5172
Epoch 15 | Train loss: 0.0160 | Test acc: 0.5632
Epoch 16 | Train loss: 0.0122 | Test acc: 0.5632
Epoch 17 | Train loss: 0.0095 | Test acc: 0.5517
Epoch 18 | Train loss: 0.0078 | Test acc: 0.5632
Epoch 19 | Train loss: 0.0065 | Test acc: 0.5517
Epoch 20 | Trai

## Modelos a usar

En mis experimentos voy a tratar cada extractor de embeddings por separado (Baseline, GaitSet, GaitPart, GaitGL, GLN-fase1, GLN-fase2, etc.) y, para cada uno, aplicaré dos tipos de pooling temporal (media y máximo) y dos esquemas de normalización (min–max y L₂). A continuación entrenaré distintos clasificadores (MLP, LSTM/BiLSTM, TCN y Transformer) con esos vectores de secuencia, registraré la precisión, el F1-score y el tiempo de entrenamiento, y guardaré los pesos de los modelos. De este modo obtendré, para cada combinación extractor–pooling–normalización–clasificador, un resultado cuantitativo que me permitirá comparar objetivamente qué pipeline ofrece el mejor rendimiento en la predicción de la dirección del penalti.

In [3]:
# 1) Definiciones de modelos
class MLP(nn.Module):
    def __init__(self, D):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(D,128), nn.ReLU(),
            nn.Linear(128,64), nn.ReLU(),
            nn.Linear(64,3)
        )
    def forward(self, x): return self.net(x)


class LSTMClassifier(nn.Module):
    def __init__(self, D, hidden=128, layers=2, bidir=False, dropout=0.5):
        super().__init__()
        self.lstm = nn.LSTM(D, hidden, layers, 
                            batch_first=True, 
                            bidirectional=bidir, 
                            dropout=dropout)
        out_dim = hidden * (2 if bidir else 1)
        self.fc = nn.Linear(out_dim, 3)
    def forward(self, x):
        # x: (B, T, D)
        _, (h_n, _) = self.lstm(x)
        h = h_n[-1]  # última capa / dirección
        return self.fc(h)
    
    
# …aquí definirías una TCN y un Transformer…


Se lleva a cabo un pipeline de experimentos para evaluar diferentes combinaciones de pooling y normalización en embeddings extraídos de vídeos. Primero, se lee el CSV y se agrupan los datos por "video_ID". Para cada grupo se aplica pooling (media o máximo) sobre las columnas de características ("feat_...") para obtener un embedding por secuencia y se extrae la etiqueta "shoot_zone". Luego, se aplica una normalización (minmax o l2) y se separan los datos en conjuntos de entrenamiento y test.

Antes de entrenar, los datos se convierten en tensores y se crean DataLoaders. Para cada combinación, se instancia un modelo clasificatorio usando la dimensión de entrada extraída con X_train.shape[1] (que representa el número total de características de cada embedding)

In [16]:
import os, pandas as pd, numpy as np
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


# 2) Pipeline de pooling + normalización
def prepare_vectors(df, pool='mean'):
    feat_cols = [c for c in df.columns if c.startswith('feat_')]
    X, y = [], []
    for vid, g in df.groupby('video_ID'):
        arr = g[feat_cols].values  # (T, D)
        seq = arr.mean(axis=0) if pool=='mean' else arr.max(axis=0)
        X.append(seq)
        y.append(int(g['shoot_zone'].iloc[0]))
    return np.vstack(X), np.array(y)

def apply_norm(X, method):
    if method=='minmax':
        return MinMaxScaler().fit_transform(X)
    elif method=='l2':
        return normalize(X, norm='l2')
    else:
        raise ValueError

# 3) Bucle de experimentos
results = []
extractors = ['baseline_CASIAB.csv', 'baseline_OUMVLP.csv']
normalizations = ['minmax','l2']
pools = ['mean','max']
classifiers = {
    'MLP': MLP,
    # 'LSTM': LSTMClassifier,
    # 'TCN': TCNClassifier,
    # 'Transformer': TransformerClassifier,
}

for csv in extractors:
    df = pd.read_csv(f'Gait_Embeddings_good/{csv}')
    for pool in pools:
      X, y = prepare_vectors(df, pool=pool)
      for norm in normalizations:
        Xn = apply_norm(X, norm)
        X_train, X_test, y_train, y_test = train_test_split(
            Xn, y, test_size=0.2, stratify=y, random_state=42
        )
        # convertir a tensores
        Xtr_t = torch.from_numpy(X_train).float()
        ytr_t = torch.from_numpy(y_train).long()
        Xte_t = torch.from_numpy(X_test).float()
        yte_t = torch.from_numpy(y_test).long()
        
        # DataLoader para MLP
        train_ds = TensorDataset(Xtr_t, ytr_t)
        train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
        test_ds  = TensorDataset(Xte_t, yte_t)
        test_dl  = DataLoader(test_ds, batch_size=32)
        
        for name, Cls in classifiers.items():
            # Instanciar modelo
            model = Cls(X_train.shape[1]).to(DEVICE) # se obtiene la D: Dimensión del embedding, es decir, numero de columnas
            opt   = optim.Adam(model.parameters(), lr=1e-3)
            lossf = nn.CrossEntropyLoss()
            
            # Entrenar (por ejemplo 20 épocas)
            for ep in range(20):
                model.train()
                for xb, yb in train_dl:
                    xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                    opt.zero_grad()
                    loss = lossf(model(xb), yb)
                    loss.backward(); opt.step()
            
            # Evaluar accuracy en test
            model.eval()
            correct = 0
            with torch.no_grad():
                for xb, yb in test_dl:
                    xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                    preds = model(xb).argmax(dim=1)
                    correct += (preds==yb).sum().item()
            acc = correct/len(test_ds)
            
            # Registrar
            results.append({
              'extractor': csv,
              'pooling': pool,
              'norm': norm,
              'classifier': name,
              'accuracy': acc
            })

# 4) Convertir results a DataFrame y comparar
import pandas as pd
df_res = pd.DataFrame(results)
print(df_res.pivot_table(
    index=['extractor','classifier'], 
    columns=['pooling','norm'], 
    values='accuracy'
))


pooling                              max                mean          
norm                                  l2    minmax        l2    minmax
extractor           classifier                                        
baseline_CASIAB.csv MLP         0.482759  0.540230  0.471264  0.482759
baseline_OUMVLP.csv MLP         0.482759  0.494253  0.494253  0.459770


In [15]:
df_res.head()

,extractor,pooling,norm,classifier,accuracy
0,baseline_CASIAB.csv,mean,minmax,MLP,0.505747
1,baseline_CASIAB.csv,mean,l2,MLP,0.505747
2,baseline_CASIAB.csv,max,minmax,MLP,0.540230
3,baseline_CASIAB.csv,max,l2,MLP,0.482759
